## TODO:

* possibly add other activation functions (non-differentiable)
* simulated annealing? nelder mead?
* right now, only globalbestpso used
* add button to run optimization only after clicking
* live plots maybe?
* Cost history / number of iterations needed compared to classic MLP

# Neural Networks

Neural networks are a way of parametrizing non-linear functions. On a very basic level, they are formed by a composition of non-linear function. The function is defined with a layered architecture. The mapping from the input layer to the output layer is performed via hidden layers. Each layer $k$ produces an output $z_k$ that is a non-linear function of a weighted combination of the outputs of the previous layer, $z_k = g_k(W_k z_{k-1})$. 

Once the architecture and the activation functions $g_k(\cdot)$ are defined, the weights $W_k$ are trained. If all the functions $g_k$ are (sub)-differentiable then, via the chain rule, gradients exist and can be computed. Usually, the weights are then trained via different variants of gradient descent.

What we do here in this notebook is another approach: instead of using a gradient based method that does the fitting of the network, we want to apply the particle swarm optimization, simulated annealing and Nelder Mead algorithms. We will use sample datasets for binary classification that are supplied by Scikit Learn.

In [1]:
import numpy as np 
import matplotlib as mpl 
import matplotlib.pyplot as plt 

from sklearn import cluster, datasets, mixture
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.model_selection import train_test_split

import ipywidgets
from ipywidgets import interact, interactive, interact_manual, fixed

from utilities import plot_helpers


%matplotlib inline
%load_ext autoreload
%autoreload 2
from matplotlib import rcParams
rcParams['figure.figsize'] = (10, 5)  # Change this if figures look ugly. 
rcParams['font.size'] = 16

import warnings
warnings.filterwarnings("ignore")

import pyswarms as ps

## Architecture

Since the PySwarm optimizer needs a specific objective function, we implement a basic neural network with that works with different activation functions and variable hidden layers. This implementation is based on NumPy.

### Activation functions

In [2]:
# Sigmoid aka. Logistic function
def sigmoid(x):
    return 1 / (1 + np.exp(-x))
# Relu function
def relu(x):
    return np.maximum(0, x)

# tanh
def tanh(x):
    return np.tanh(x)

def inv(x):
    return np.sqrt(np.abs(x))

activations = {
    "logistic": sigmoid,
    "relu": relu,
    "identity": lambda x: x,
    "tanh": tanh,
    "inv": inv
}

In [3]:
n_samples = 200
n_features = 2
n_classes = 2

## Forward Prop, Loss and Prediction

In [4]:
def forward_prop(p, activation, hidden_layer_sizes, X):
    """ Calculate roll-back the weights and biases
    Inputs
    ------
    p: np.ndarray
        unrolled version of the weights and biases
        
    activation: function :: np.ndarray -> np.ndarray
  
    hidden_layer_sizes: tuple (int)
  
    X: np.ndarray

    Returns
    -------
    numpy.ndarray of logits for the output layer, which correspond
    to the probabilities of predicting each class

    """
    n_inputs = n_features
    start = 0
    in_i = X
    for i in range(0, len(hidden_layer_sizes)):
        layer_size = hidden_layer_sizes[i]
        no_weights = n_inputs * layer_size
        # get weights and biases for this layer
        W_i = p[start:(start+no_weights)].reshape((n_inputs, layer_size))
        b_i = p[(start+no_weights):(start+no_weights+layer_size)].reshape((layer_size,))
        
        out_i = activation(in_i.dot(W_i) + b_i)
        
        # update variables
        start = start + no_weights + layer_size
        n_inputs = layer_size
        in_i = out_i
    
    # Compute last layer
    no_weights = n_inputs * n_classes
    W_k = p[start:(start+no_weights)].reshape((n_inputs, n_classes))
    b_k = p[(start+no_weights):(start+no_weights+n_classes)].reshape((n_classes,))
    
    # Pre-activation
    out = in_i.dot(W_k) + b_k
    
    # Probabilities using softmax
    # make every value 0 or below, as exp(0) won't overflow
    out_scaled = out - out.max(axis=-1, keepdims=True)
    exp_scores = np.exp(out_scaled)
    return exp_scores / np.sum(exp_scores, axis=1, keepdims=True)

In [5]:
# Getting the loss of our current network
def loss(params, activation, hidden_layer_sizes, reg, X, Y):
    probs = forward_prop(params, activation, hidden_layer_sizes, X)
    correct_logprobs = -np.log(probs[range(X.shape[0]), Y])
    loss = np.sum(correct_logprobs) / X.shape[0]
    return loss + reg * np.linalg.norm(params)

In [6]:
# Prediction using the probabilities of our forward propagation
def predict(params, activation, hidden_layer_sizes, X):
    probs = forward_prop(params, activation, hidden_layer_sizes, X)
    y_pred = np.argmax(probs, axis=1)
    return y_pred

## Classification Demo

Neural network training has a lot of hyperparameters. Architecture, learning rate, batch size, optimization algorithm, random seed are just a few of them. Additionally, we have the hyperparameters for the swarm optimization. These generally are:
* $c_1$, the cognitive parameter (attraction towards individual best)
* $c_2$, the social parameter (attraction towards global/neighborhood best)
* $w$, the inertia or momentum

Also, the topology can be specified when using the local optimization method (using neighborhood best).

#### Data creation

In [7]:
def data_creation(dataset, noise):
    if dataset is 'blobs':
        X, Y = datasets.make_blobs(n_samples=n_samples, centers=2, random_state=3, cluster_std=10*noise)
    elif dataset is 'circles':
        X, Y = datasets.make_circles(n_samples=n_samples, factor=.5, noise=noise, random_state=42)
    elif dataset is 'moons':
        X, Y = datasets.make_moons(n_samples=n_samples, noise=noise, random_state=42)
    elif dataset == 'xor':
        np.random.seed(42)
        step = int(n_samples/4)
        
        X = np.zeros((n_samples, 2))
        Y = np.zeros(n_samples)
        
        X[0*step:1*step, :] = noise * np.random.randn(step, 2)
        Y[0*step:1*step] = 1
        X[1*step:2*step, :] = np.array([1, 1]) + noise * np.random.randn(step, 2)
        Y[1*step:2*step] = 1
        
        X[2*step:3*step, :] = np.array([0, 1]) + noise * np.random.randn(step, 2)
        Y[2*step:3*step] = 0
        X[3*step:4*step, :] = np.array([1, 0]) + noise * np.random.randn(step, 2)
        Y[3*step:4*step] = 0
    
    elif dataset == 'periodic':
        
        step = int(n_samples/4)
        
        X = np.zeros((n_samples, 2))
        Y = np.zeros(n_samples)
        
        X[0*step:1*step, :] = noise * np.random.randn(step, 2)
        Y[0*step:1*step] = 1
        X[1*step:2*step, :] = np.array([0, 2]) + noise * np.random.randn(step, 2)
        Y[1*step:2*step] = 1
        
        X[2*step:3*step, :] = np.array([0, 1]) + noise * np.random.randn(step, 2)
        Y[2*step:3*step] = 0
        X[3*step:4*step, :] = np.array([0, 3]) + noise * np.random.randn(step, 2)
        Y[3*step:4*step] = 0
    
    
    X = X[Y <= 1, :]
    Y = Y[Y <=1 ]
   # Y[Y==0] = -1
    return X, Y

#### Defining the interactive function

In [8]:
rcParams['figure.figsize'] = (10, 5)  # Change this if figures look ugly. 
rcParams['font.size'] = 16
def mlp(solver, k, dataset, hidden_layer_sizes, activation, iters, particles, c1, c2, w, reg, noise):
    np.random.seed(42)
    
    # Initialize swarm
    
    # to calculate the dimension, we know we have (in * out) no. of weights
    # and (out) no. of biases per layer
    dimensions = 0
    in_i = n_features
    for l in hidden_layer_sizes:
        dimensions += in_i * l + l
        in_i = l
    dimensions += in_i * n_classes + n_classes
    print("Dimensions for this problem:", dimensions)
    
    
    # The options for the optimizer
    options = {'c1': c1, 'c2': c2, 'w': w, 'k': k, 'p': 2}
    
    # Pick optimizer
    if solver == 'GlobalBest':
        optimizer = ps.single.GlobalBestPSO(n_particles=particles, dimensions=dimensions,\
                                        options=options)
        
    elif solver == 'LocalBest':        
        optimizer = ps.single.LocalBestPSO(n_particles=particles, dimensions=dimensions,\
                                           options=options)
    # get function to corresponding string
    activation_function = activations[activation]
    
    # get the data
    X, Y = data_creation(dataset, noise)
    
    X = StandardScaler().fit_transform(X)
    X_train, X_test, y_train, y_test = train_test_split(X, Y.astype(int), test_size=.2)

    # wrap our function 
    def f(x):
        n_particles = x.shape[0]
        res = [loss(x[i], activation_function, hidden_layer_sizes, 10**reg, X_train, y_train)\
               for i in range(n_particles)]
        return np.array(res)
    
    cost, pos = optimizer.optimize(f, iters=iters)
    print((y_test == predict(pos, activation_function, hidden_layer_sizes, X_test)).mean())
    
    
    # plot the line, the points, and the nearest vectors to the plane
    plt.figure()
    plt.clf()
    fig = plt.axes()
    opt = {'marker': 'r*', 'label': '+'}
    plot_helpers.plot_data(X[np.where(Y == 1)[0], 0], X[np.where(Y == 1)[0], 1], fig=fig, options=opt)
    opt = {'marker': 'bs', 'label': '-'}
    plot_helpers.plot_data(X[np.where(Y == 0)[0], 0], X[np.where(Y == 0)[0], 1], fig=fig, options=opt)

    mins = np.min(X, 0)
    maxs = np.max(X, 0)
    x_min = mins[0] - 1
    x_max = maxs[0] + 1
    y_min = mins[1] - 1
    y_max = maxs[1] + 1

    XX, YY = np.mgrid[x_min:x_max:200j, y_min:y_max:200j]  
    Xplot = np.c_[XX.ravel(), YY.ravel()]   
    
    # get all the predictions
    Z = forward_prop(pos, activation_function, hidden_layer_sizes, Xplot)[:,1]
    
    # Put the result into a color plot
    Z = Z.reshape(XX.shape)
    # plt.figure(fignum, figsize=(4, 3))
    # Put the result into a color plot
    plt.contourf(XX, YY, Z, cmap=plt.cm.jet, alpha=.3)

### Interactive modelation

To be able to visually see and play around with the different parameters, we here give an interactive tool. One can pick different hyperparameters, datasets, layers etc. and see the dataset as well as the decision boundaries of the trained weights and biases of the trained network.

In [9]:
solver_widget = ipywidgets.Dropdown(options=['GlobalBest', 'LocalBest'],\
                            value='GlobalBest', description='Solver', disabled=False)

particles_widget = ipywidgets.Dropdown(options=[20,50,100,200,1000],\
                               value=20, description='Particles', disabled=False)

k_widget = ipywidgets.IntSlider(value=5,
                            min=1,
                            max=particles_widget.value,
                            step=1,
                            readout_format='d',
                            description='neighborhood k:',
                            style={'description_width': 'initial'},
                            continuous_update=False,
                            disabled=True)

def show_k(*args):
    k_widget.disabled = True if solver_widget.value != 'LocalBest' else False
    k_widget.max = particles_widget.value

particles_widget.observe(show_k)
solver_widget.observe(show_k)

interact_manual(mlp, 
        solver=solver_widget,
        k=k_widget,
        dataset=['blobs', 'circles', 'moons', 'xor', 'periodic'],
        activation=['relu', 'logistic', 'identity', 'tanh', 'inv'],
        hidden_layer_sizes=[(50, ), (100, ), (50, 50), (100, 100), (50, 50, 50), (100, 100, 100)],
        iters=[100,200,500,1000,5000],
        particles=particles_widget,
        c1=ipywidgets.FloatSlider(value=0.5,
                                    min=0,
                                    max=1,
                                    step=0.1,
                                    readout_format='.1f',
                                    description='c1:',
                                    style={'description_width': 'initial'},
                                    continuous_update=False),
        c2=ipywidgets.FloatSlider(value=0.3,
                                    min=0,
                                    max=1,
                                    step=0.1,
                                    readout_format='.1f',
                                    description='c2:',
                                    style={'description_width': 'initial'},
                                    continuous_update=False),
        w=ipywidgets.FloatSlider(value=0.9,
                                    min=0,
                                    max=1,
                                    step=0.1,
                                    readout_format='.1f',
                                    description='w:',
                                    style={'description_width': 'initial'},
                                    continuous_update=False),
        reg=ipywidgets.FloatSlider(value=-3,
                                    min=-3,
                                    max=3,
                                    step=0.1,
                                    readout_format='.1f',
                                    description='reg 10^:',
                                    style={'description_width': 'initial'},
                                    continuous_update=False),
        noise=ipywidgets.FloatSlider(value=0.05,
                                    min=0.01,
                                    max=0.3,
                                    step=0.01,
                                    readout_format='.2f',
                                    description='noise:',
                                    style={'description_width': 'initial'},
                                    continuous_update=False),  
        );

interactive(children=(Dropdown(description='Solver', options=('GlobalBest', 'LocalBest'), value='GlobalBest'),…

In fact, we can see that the PSO works quite well to find the fit for the classification and judging by the visualization the decision boundaries found by the swarm is very much reasonable in almost all cases. Evidently, the underlying function we are trying to optimize is an extremely high-dimensional non-convex function with (in general) no single global optimum, but many local minimas or possible solutions.

Interestingly, the PSO can give somewhat different results compared to popular optimization methods like SGD and Adam. In fact, using the logistic activation function for a linearly non-separable dataset likesmoons, classic SGD fails to find a good solution, whereas PSO does.


![PSO vs. SGD for logistic activation](logistic_pso_sgd.png)
*PSO vs. SGD for logistic activation and the moon dataset, one hidden layer*

### Performance

The biggest problem with PSO and neural network training certainly is the complexity. With just one hidden layer of size $50$, we have $252$ weights and biases to be adjusted, therefore a $252$-dimensional function which we then evaluate $50 * 200 = 10^4$ times for e.g. $50$ particles and $200$ iterations.
<br/><br/>

Let's consider this with a small excursion to the theoretical complexity of the forward propagation:

Given one feature vector $x$, we first multiply the vector with the weight matrix $W_1$, and then add the bias $b_1$ before applying the activation function. Assuming the number of perceptrons in the hidden layer have the same dimension $n$ like the input, we have a complexity of $O(n^2)$ as a result of the matrix with vector multiplication. Repeating this for $k$ layers and $m$ data points, we get something in the order of $O(k*m*n^2)$. Repeating this operation for every particle for every iteration therefore is a big computational burden.
<br/> <br/> <br/>
Let's have a comparison for one specific example:

## Non-differentiable activation functions
Some typical properties one looks for in an activation function are:
* Non-linearity (to be able to learn arbitrary functions)
* Monotonicity and "Smoothness"
* Close approximation of the identity function near the origin (helps with learning after random initialization)
* **Continous differentiability**

The commonly used activation functions for neural networks, like $sigmoid$ and $tanh$ seen above, are differentiable in order to allow gradient-based optimization methods. A very popular activation function that is actually not differentiable (at 0) is $ReLU(x)=max(x,0)$. However, it is in fact the most popular activation function because of its simplicity and robustness to the vanishing gradient problem.

Now that we are using optimization methods like the PSO, we do not need to require any differentiability and can, in fact, create arbitrary complex functions as our activations. Of course, it is nevertheless important to have functions that can be efficiently *evaluated*.

In [10]:
# step function, where return value is 
# 0 for negative, 1 for positive, and constant (chosen to be 1 here) for 0

def heaviside(x):
    return np.heaviside(x, 1)